In [ ]:
from googleapiclient.discovery import build
from dateutil import parser
import pandas as pd
from IPython.display import JSON
from datetime import datetime, timedelta
import isodate #parse
import os
import numpy as np
import csv

# Data viz packages
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

# NLP
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud

In [ ]:
api_key = 'your_API_key'

In [ ]:
channel_id='your_channel_id'

In [ ]:
api_service_name = "youtube"
api_version = "v3"

# Get credentials and create an API client
youtube = build(api_service_name, api_version, developerKey=api_key)

In [ ]:
# channelName, subscribers count, total views, totalVideos, playlistId

get_channel_stats(youtube, channel_ids):
    
    all_data = []
    request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id=','.join(channel_id))
    
    response = request.execute()

    # loop through items
    for item in response['items']: # Some channels have "0" items
        data = {'channelName': item['snippet']['title'],
                'subscribers': item['statistics']['subscriberCount'],
                'views': item['statistics']['viewCount'],
                'totalVideos': item['statistics']['videoCount'],
                'playlistId': item['contentDetails']['relatedPlaylists']['uploads']
        }
        
        # Check if the channel has any subscriber milestone
        if 'bulletin' in item['contentDetails']:
            milestone = item['contentDetails']['bulletin']['resource']
            if milestone['kind'] == 'youtube#subscription':
                data['milestoneDate'] = milestone['publishedAt']
                data['milestoneSubscribers'] = milestone['metadata']['subscriberCount']
                
        all_data.append(data)
        
    return(pd.DataFrame(all_data))

In [ ]:
channel_stats = get_channel_stats(response)
channel_stats

In [ ]:
# ids

playlist_id = "your_channel_id"

def get_video_ids(youtube, playlist_id):
    
    video_ids = []
    
    request = youtube.playlistItems().list(
        part="snippet, contentDetails",
        playlistId=playlist_id,
        maxResults = 50
    )
    response = request.execute()

    for item in response['items']:
        video_ids.append(item['contentDetails']['videoId'])
        
    next_page_token = response.get('nextPageToken')
    while next_page_token is not None:
        request = youtube.playlistItems().list(
            part="snippet, contentDetails",
            playlistId=playlist_id,
            maxResults = 50,
            pageToken = next_page_token)
   
        response = request.execute()

        for item in response['items']:
            video_ids.append(item['contentDetails']['videoId'])

        next_page_token = response.get('nextPageToken')

    return video_ids

In [ ]:
# Get Video IDs
video_ids=get_video_ids(youtube, playlist_id)

In [ ]:
# the list of videos; channelTitle, title, ...

def get_video_details(youtube, video_ids):

    all_video_info = []

    for i in range(0, len(video_ids), 50):
        request = youtube.videos().list(
            part="snippet, contentDetails, statistics",
            id=','.join(video_ids[i:i+50])
        )
        response = request.execute()

        for video in response['items']:
            stats_to_keep = {'snippet': ['channelTitle', 'title', 'description', 'tags', 'publishedAt'],
                             'statistics': ['viewCount', 'likeCount', 'favouriteCount', 'commentCount'],
                             'contentDetails': ['duration', 'definition', 'caption']
                            }
            video_info={}
            video_info['video_id']=video['id']

            for k in stats_to_keep.keys():
                for v in stats_to_keep[k]:
                    try:
                        video_info[v] = video[k][v]
                    except:
                        video_info[v]=None

            all_video_info.append(video_info)
    
    return pd.DataFrame(all_video_info)

In [ ]:
# Get video details 
df = get_video_details(youtube, video_ids)

## Data pre-processing

In [ ]:
video_df = df.copy()
video_df.info()

In [ ]:
# Publish day in the week
video_df['publishedAt'] = video_df['publishedAt'].apply(lambda x: parser.parse(x)) 
video_df['pushblishDayName'] = video_df['publishedAt'].apply(lambda x: x.strftime("%A"))

# convert duration to seconds
video_df['durationSecs'] = video_df['duration'].apply(lambda x: isodate.parse_duration(x))
video_df['durationSecs'] = video_df['durationSecs'].astype('timedelta64[s]')

# Create Month & Year columns
video_df['Month'] = pd.to_datetime(video_df["publishedAt"]).dt.strftime('%b')
video_df['Year'] = pd.to_datetime(video_df['publishedAt']).dt.strftime('%b')
video_df['Year'] = video_df['publishedAt'].dt.year

# Create Mins & Hours columns for the duration
video_df["durationMins"] = (video_df["durationSecs"]/60).round(2).astype(float)
video_df["durationHours"] = (video_df["durationMins"]/60).round(2).astype(float)

# Add tag count
video_df['tagCount'] = video_df['tags'].apply(lambda x: 0 if x is None else len(x))

In [ ]:
# Convert invalid values to NaN
video_df = video_df.replace({'viewCount': {'None': np.nan}, 
                              'likeCount': {'None': np.nan}, 
                              'commentCount': {'None': np.nan}})

# Drop rows with NaN values
video_df.dropna(subset=['viewCount', 'likeCount', 'commentCount'], inplace=True)

# Convert columns to int type
video_df = video_df.astype({"viewCount": int, "likeCount": int, "commentCount": int})

In [ ]:
# Remove the particular columns for further analysis
column_names = video_df.columns.values.tolist()
columns_removed = ["favouriteCount","caption","duration","definition","video_id","tags", "description","tagCount"]

# create new list using list comprehension
column_names = [i for i in column_names if i not in columns_removed]

print(column_names)

In [ ]:
file_name = video_df["channelTitle"].unique().tolist()
joined = ('_or_').join(file_name).replace(' ', '_').lower()

# Create directory
dirName = f"/Users/your_folder/{joined}"

# Create target Directory if don't exist
if not os.path.exists(dirName):
    os.mkdir(dirName)
    print("Directory " , dirName ,  " Created ")
else:    
    print("Directory " , dirName ,  " already exists")

try:
    # Create target Directory
    os.mkdir(dirName)
    print("Directory " , dirName ,  " Created ") 
except FileExistsError:
    print("Directory " , dirName ,  " already exists")

In [ ]:
# Create path and save df as csv
path = f"/Users/your_folder/{joined}/stats_{joined}.csv"
video_df.to_csv(path, index=False, columns=column_names)

In [ ]:
# Save df as csv
video_df_before =video_df[video_df["publishedAt"]< "2023-02-06"]

path2= f"/Users/your_folder/media_stats/{joined}/{joined}_b4.csv"
video_df_before.to_csv(path2, index=False, columns=column_names)

In [ ]:
# Save df as csv
video_df_after =video_df[video_df["publishedAt"]>= "2023-02-06"]

path3= f"/Users/your_folder/media_stats/{joined}/{joined}_aft.csv"
video_df_after.to_csv(path3, index=False, columns=column_names)

## Feature Engineering

### Ratios

In [ ]:
# Video subscribers/video ratio
channel_stats["subscribers_per_video"] = round((channel_stats["subscribers"])/(channel_stats["totalVideos"]),0)

# Video view/video ratio
channel_stats["view_per_video"] = round((channel_stats["views"])/channel_stats["totalVideos"],0)

In [ ]:
# number of the current month 
today = datetime.now()
current_month = int(today.strftime("%m"))
print("Current Month with Decimal Number :", current_month);

#### Total - Monthly Video Uploads after

In [ ]:
video_after_per_month = video_df_after.groupby('Month', as_index =False).size()
sort_order=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec',]
video_after_per_month.index = pd.CategoricalIndex(video_after_per_month['Month'], categories = sort_order, ordered= True)

video_after_per_month.reset_index(drop=True, inplace=True)
video_after_per_month = video_after_per_month.rename(columns={'size': 'monthly_video_count_after'})
video_after_per_month

#### Total - Upload Mins after

In [ ]:
mins_after_per_month = video_df_after.groupby(["Year","Month"])['durationMins'].sum().unstack(level='Year').replace('nan', np.nan).fillna(0)
mins_after_per_month = pd.DataFrame(data=mins_after_per_month).reset_index().set_axis(['Month', 'mins_after_per_month'], axis=1)
mins_after_per_month

merged_df1 = video_after_per_month.merge(mins_after_per_month, left_on='Month', right_on='Month')
merged_df1

#### Total - Video likes after

In [ ]:
likes_after_per_month = video_df_after.groupby(["Year","Month"])['likeCount'].sum().unstack(level='Year').replace('nan', np.nan).fillna(0)
likes_after_per_month = pd.DataFrame(data=likes_after_per_month).reset_index().set_axis(['Month', 'likes_after_per_month'], axis=1)
likes_after_per_month

merged_df2 = merged_df1.merge(likes_after_per_month, left_on='Month', right_on='Month')
merged_df2

#### Total Video views after

In [ ]:
views_after_per_month = video_df_after.groupby(["Year","Month"])['viewCount'].sum().unstack(level='Year').replace('nan', np.nan).fillna(0)
views_after_per_month = pd.DataFrame(data=views_after_per_month).reset_index().set_axis(['Month', 'views_after_per_month'], axis=1)
views_after_per_month

merged_df3 = merged_df2.merge(views_after_per_month, left_on='Month', right_on='Month')
merged_df3

#### Total - Video comments after

In [ ]:
comments_after_per_month = video_df_after.groupby(["Year","Month"])['commentCount'].sum().unstack(level='Year').replace('nan', np.nan).fillna(0)
comments_after_per_month = pd.DataFrame(data=comments_after_per_month).reset_index().set_axis(['Month', 'comments_after_per_month'], axis=1)
comments_after_per_month

merged_df4 = merged_df3.merge(comments_after_per_month, left_on='Month', right_on='Month')
merged_df4

#### Totals - merged & saved

In [ ]:
merged_df4["channelName"] = ""
merged_df4['channelName'] = merged_df4['channelName'].replace([''], channel_stats["channelName"].tolist())
merged_df4.reset_index(drop=True, inplace=True)

# Define the new row values
new_row = merged_df4.loc[0:2,:].values.tolist()

# Open the existing CSV file for reading
with open('/Users/your_folder/monthly_totals.csv', 'r') as file:
    reader = csv.reader(file)
    rows = list(reader)

# Add the new row to the list of rows
rows.extend(new_row)

# Open the CSV file for writing and write the updated rows
with open('/Users/your_folder/monthly_totals.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(rows)

### Ratios after 6th of Feb, 2023

In [ ]:
# Video uploaded after 6th of Feb, 2023
video_count_after = video_df_after["title"].count()
channel_stats["video_count_after"] = video_count_after # Checked

# Video durationSecs after 6th of Feb, 2023
secs_count_after = int(video_df_after["durationSecs"].sum())
channel_stats["secs_count_after"] = secs_count_after # Checked

# Video durationMins after 6th of Feb, 2023
mins_count_after = round(video_df_after["durationMins"].sum(),1)
channel_stats["mins_count_after"] = mins_count_after # Checked

# Video durationHours after 6th of Feb, 2023
hours_count_after = round(video_df_after["durationHours"].sum(),1)
channel_stats["hours_count_after"] = hours_count_after # Checked

# Video views after 6th of Feb, 2023
view_count_after = video_df_after["viewCount"].sum()
channel_stats["view_count_after"] = view_count_after # Checked

# Video likes after 6th of Feb, 2023
like_count_after = video_df_after["likeCount"].sum()
channel_stats["like_count_after"] = like_count_after # Checked

# Video comments after 6th of Feb, 2023
comment_count_after = video_df_after["commentCount"].sum()
channel_stats["comment_count_after"] = comment_count_after # Checked

# Video comment/video ratio after 6th of Feb, 2023
comment_per_video_after = round(comment_count_after/video_count_after,0)
channel_stats["comment_per_video_after"] = comment_per_video_after

# Video view/video ratio after 6th of Feb, 2023
view_per_video_after = round(view_count_after/video_count_after,0)
channel_stats["view_per_video_after"] = view_per_video_after

# Video like/video ratio after 6th of Feb, 2023
like_per_video_after = round(like_count_after/video_count_after,0)
channel_stats["like_per_video_after"] = like_per_video_after

# Video secs/video ratio after 6th of Feb, 2023
secs_per_video_after = round(secs_count_after/video_count_after,1)
channel_stats["secs_per_video_after"] = secs_per_video_after

# Video mins/video ratio after 6th of Feb, 2023
mins_per_video_after = round(mins_count_after/video_count_after,1)
channel_stats["mins_per_video_after"] = mins_per_video_after

# Video hours/video ratio after 6th of Feb, 2023
hours_per_video_after = round(hours_count_after/video_count_after,1)
channel_stats["hours_per_video_after"] = hours_per_video_after

# Video like/comment ratio after 6th of Feb, 2023
channel_stats["like_per_comment_after"] = round(like_count_after/comment_count_after,1)

# Video view/like ratio after 6th of Feb, 2023
channel_stats["view_per_like_after"] = round(view_count_after/like_count_after,1)

channel_stats

### Ratios before 6th of Feb, 2023

In [ ]:
# Video uploaded before 6th of Feb, 2023
video_count_before = int(channel_stats["totalVideos"]) - video_count_after
channel_stats["video_count_before"] = video_count_before # Checked

# Video views before 6th of Feb, 2023
view_count_before = int(channel_stats["views"]) - view_count_after
channel_stats["view_count_before"] = view_count_before # Checked

# Video view/video ratio before 6th of Feb, 2023
view_per_video_before = round(view_count_before/video_count_before,1)
channel_stats["view_per_video_before"] = view_per_video_before # Checked

# channel_stats.reset_index(drop=True, inplace=True)
channel_stats

In [ ]:
column_names_new = channel_stats.columns.values.tolist()
channel_stats.reset_index(drop=True, inplace=True)

# Define the new row values
new_row = channel_stats.loc[0,:].values.tolist()

# Open the existing CSV file for reading
with open('/Users/your_folder/media_stats_edited.csv', 'r') as file:
    reader = csv.reader(file)
    rows = list(reader)

# Add the new row to the list of rows
rows.append(new_row)

# Open the CSV file for writing and write the updated rows
with open('/Users/your_folder/media_stats_edited.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(rows)

## EDA

### Best Performing Videos

In [ ]:
sns.set(rc = {'figure.figsize':(10,6)})
ax = sns.barplot(x = 'title', y = 'viewCount', data = video_df.sort_values('viewCount', ascending=False)[0:9])
plot = ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos:'{:,.0f}'.format(x/1000) + 'K'))

In [ ]:
sns.set(rc = {'figure.figsize':(10,6)})
ax = sns.barplot(x = 'title', y = 'viewCount', data = video_df_before.sort_values('viewCount', ascending=False)[0:9])
plot = ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
ax.set(title='After Earthquake 6th Feb 2023')
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos:'{:,.0f}'.format(x/1000) + 'K'))

In [ ]:
sns.set(rc = {'figure.figsize':(10,6)})
ax = sns.barplot(x = 'title', y = 'viewCount', data = video_df_before.sort_values('viewCount', ascending=False)[0:9])
plot = ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
ax.set(title='Before Earthquake 6th Feb 2023')
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos:'{:,.0f}'.format(x/1000) + 'K'))

### Worst performing videos

In [ ]:
ax = sns.barplot(x = 'title', y = 'viewCount', data = video_df.sort_values('viewCount', ascending=True)[0:9])
plot = ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
ax.bar_label(ax.containers[0], label_type='edge')
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos:'{:,.0f}'.format(x/1000) + 'K'))

In [ ]:
ax = sns.barplot(x = 'title', y = 'viewCount', data = video_df_after.sort_values('viewCount', ascending=True)[0:9])
plot = ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
ax.bar_label(ax.containers[0], label_type='edge')
ax.set(title='After Earthquake 6th Feb 2023')
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos:'{:,.0f}'.format(x/1000) + 'K'))

In [ ]:
ax = sns.barplot(x = 'title', y = 'viewCount', data = video_df_before.sort_values('viewCount', ascending=True)[0:9])
plot = ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
ax.bar_label(ax.containers[0], label_type='edge')
ax.set(title='Before Earthquake 6th Feb 2023')
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos:'{:,.0f}'.format(x/1000) + 'K'))

### View distribution per video

In [ ]:
sns.violinplot(video_df['channelTitle'], video_df['viewCount'])
plt.show()

In [ ]:
sns.violinplot(video_df_after['channelTitle'], video_df_after['viewCount']).set(title='After Earthquake 6th Feb 2023')
plt.show()

In [ ]:
sns.violinplot(video_df_before['channelTitle'], video_df_before['viewCount']).set(title='Before Earthquake 6th Feb 2023')
plt.show()

### Views vs. likes and comments

In [ ]:
ax = sns.scatterplot(x='likeCount', y='viewCount',hue= 'commentCount', data= video_df)
ax.ticklabel_format(style='plain')
# ax.get_yaxis().set_visible(False)
plt.show()

#### After Earthquake on 6th Feb 2023

In [ ]:
ax = sns.scatterplot(x='likeCount', y='viewCount',hue= 'commentCount', data= video_df_after)
ax.ticklabel_format(style='plain')
# ax.get_yaxis().set_visible(False)
plt.show()

#### Before Earthquake on 6th Feb 2023

In [ ]:
ax = sns.scatterplot(x='likeCount', y='viewCount',hue= 'commentCount', data= video_df_before)
ax.ticklabel_format(style='plain')
# ax.get_yaxis().set_visible(False)
plt.show()

### Video duration

In [ ]:
sns.histplot(data = video_df, x = 'durationSecs', bins=100)
plt.show()

In [ ]:
sns.histplot(data = video_df_before[video_df_before["durationSecs"]<50000], x = 'durationSecs', bins=100).set(title='Before Earthquake 6th Feb 2023')
plt.show()

In [ ]:
sns.histplot(data = video_df_after, x = 'durationSecs', bins=100).set(title='After Earthquake 6th Feb 2023')
plt.show()

### Wordcloud for video titles

#### After Earthquake 6th Feb 2023

In [ ]:
stop_words = set(stopwords.words('english'))
video_df_after['title_no_stopwords'] = video_df_after['title'].apply(lambda x: [item for item in str(x).split() if item not in stop_words])

all_words = list([a for b in video_df_after['title_no_stopwords'].tolist() for a in b])
all_words_str = ' '.join(all_words) 

def plot_cloud(wordcloud):
    plt.figure(figsize=(30, 20))
    plt.imshow(wordcloud)
    plt.axis("off");

wordcloud = WordCloud(width = 2000, height = 1000, random_state=1, background_color='black', 
                      colormap='viridis', collocations=False).generate(all_words_str)
plot_cloud(wordcloud)

### Upload schedule

#### Daily Upload After

In [ ]:
import datetime

# Set the specific date and time
start_date = datetime.datetime(2023, 2, 6, 4, 17, 0)  # Year, Month, Day, Hour, Minute, Second
end_date =datetime.datetime.now()

# Define the days of the week to count
days_of_week = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

# Initialize an empty dictionary to store the counts
counts = {}

# Loop through the days of the week and count the number of occurrences
for day in days_of_week:
    count = 0
    current_date = start_date
    while current_date <= end_date:
        if current_date.strftime("%A") == day:
            count += 1
        current_date += datetime.timedelta(days=1)
    counts[day] = count

# Convert the counts to a pandas dataframe and display the result
count_days_after = pd.DataFrame.from_dict(counts, orient="index", columns=["pushblishDayName"])
count_days_after = count_days_after.reset_index().set_axis(['pushblishDayName', 'count'], axis=1)
count_days_after

In [ ]:
avg_daily_video_after = pd.DataFrame(video_df_after['pushblishDayName'].value_counts()).reset_index().set_axis(['pushblishDayName', 'video_daily_after'], axis=1)
avg_daily_video_after = avg_daily_video_after.merge(count_days_after, how='left', on='pushblishDayName')
avg_daily_video_after["daily_counts"] = round((avg_daily_video_after["video_daily_after"] / avg_daily_video_after["count"]),1)

# create a categorical data type for the days of the week
cat_type = pd.api.types.CategoricalDtype(categories=days_of_week, ordered=True)
avg_daily_video_after['pushblishDayName'] = avg_daily_video_after['pushblishDayName'].astype(cat_type)

# sort the dataframe based on the categorical data type
avg_daily_video_after = avg_daily_video_after.sort_values('pushblishDayName')

avg_daily_video_after.reset_index(drop=True, inplace=True)

avg_daily_video_after["channelName"] = ""
avg_daily_video_after['channelName'] = avg_daily_video_after['channelName'].replace([''], channel_stats["channelName"].tolist())
avg_daily_video_after

In [ ]:
avg_daily_video_after.reset_index(drop=True, inplace=True)

# Define the new row values
new_row = avg_daily_video_after.loc[0:7,:].values.tolist()

# Open the existing CSV file for reading
with open('/Users/gulecs/Desktop/youtube api/media_stats/daily_totals.csv', 'r') as file:
    reader = csv.reader(file)
    rows = list(reader)

# Add the new row to the list of rows
rows.extend(new_row)

# Open the CSV file for writing and write the updated rows
with open('/Users/gulecs/Desktop/youtube api/media_stats/daily_totals.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(rows)

In [ ]:
ax = avg_daily_video_after.plot.bar(x='pushblishDayName', y='daily_counts', rot=0, xlabel="")
ax.bar_label(ax.containers[0], label_type='edge')
ax.set(title='After Earthquake 6th Feb 2023')
ax.get_legend().remove()
plt.show()

#### Monthly Upload After

In [ ]:
ax2_1 = sns.barplot(x='Month', y='monthly_video_count_after', data=video_after_per_month, order=sort_order)
ax2_1.bar_label(ax2_1.containers[0], label_type='edge')
ax2_1.set(title='After Earthquake 6th Feb 2023')
plt.show()